# Kompression

## Utility Klassen

Wir beginnen mit einigen Klassen, die wir in früheren Notebooks eingeführt haben, in diesem Zusammenhang aber wieder brauchen. 

In [ ]:
class Alphabet:
    
    def __init__(self, chars):
        self._inverse = [None]*len(chars)
        
        self._alphabet=dict()
        for (i, c) in enumerate(chars):
            self._alphabet[c] = i
            self._inverse[i] = c
    
    def toChar(self, index):
        return self._inverse[index]
        
    def toIndex(self, char):
        return self._alphabet[char]
    
    def contains(self, char):
        return self._alphabet.contains(char)
        
    def radix(self):
        return len(self._inverse)
    
lowercase = Alphabet("abcdefghijklmnopqrstuvwxyz")
uppercase = Alphabet("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
ascii = Alphabet([chr(i) for i in range(0, 128)])
extendedAscii = Alphabet([chr(i) for i in range(0, 256)])
DNA = Alphabet("ACGT")  

In [ ]:
from graphviz import Graph

def plotTrie(root):
    dot = Graph(comment='Tree')
    _plotTrie(root, None, dot, '')
    display(dot)

def _plotTrie(node, parent, dot, edgeLabel):                         
    if node == None:        
        return 

    if node.char != None:
        dot.node(str(id(node)), str(node.char))
    else:
        dot.node(str(id(node)), "")
        
    if parent != None:
        dot.edge(str(id(parent)), str(id(node)), edgeLabel)    
    
    _plotTrie(node.left, node, dot, '0')
    _plotTrie(node.right, node, dot, '1')

        


## Binärer Input  / Output

Als nächstes führen wir die Klasse ```BinaryStream``` ein, die uns erlaubt Zeichen und Zahlen als Binärstream zu schreiben. Intern verwenden wir dafür eine Queue (deque in Python) von Booleans. 

In [ ]:
from collections import deque
class BinaryStream:
    def __init__(self):
        self._data = deque()
        self._alphabet = extendedAscii
    
    def writeBit(self, b):
        if bool(b):
            self._data.append("1")
        else:
            self._data.append("0")
        
    def writeChar(self, c):
        self.writeNBitNumber(self._alphabet.toIndex(c), 8)
        
    def writeString(self, s):
        for c in s:
            self.writeChar(c)
      
    def writeNBitNumber(self, number, numBits):
        bitString = '{0:0>{n}b}'.format(number, n = numBits)
        for b in bitString:
            self.writeBit(b == "1")

    def writeInt(self, number):
        self.writeNBitNumber(number, 32)

    def readBit(self):
        return self._data.popleft() == "1"
    
      
    def readChar(self):
        return self._alphabet.toChar(self.readNBitNumber(8))
    
    
    def readString(self, nChars = -1):
        s = ""
        numRead = nChars if nChars != -1 else len(self._data)
        while numRead < nChars and not self.isEmpty():
            s += self.readChar()
            numRead += 1
        return s
          
    def readNBitNumber(self, numBits):
        number = 0
        for i in range(numBits, 0, -1):
            if self.readBit() == True:
                number += 2**(i-1)
        return number
    
    def readInt(self):
        return self.readNBitNumber(32)
    
    def isEmpty(self):
        return self.length() == 0
    
    def length(self):
        return len(self._data)
    
    def __str__(self):
        s = ""
        for c in self._data:
            s += c        
        return s
    
    def asHexString(self):
        if self.isEmpty():
            return ""
        else:
            return hex(int(str(self), 2))
    
    def __iter__(self):
        for b in self._data:
            yield b == "1"


## Beispiel: Codieren des Datums

Im Folgenden codieren wir ein Datum auf drei verschiedene Arten. Wir verwenden dabei unsere Bitstream Klasse. Dabei sehen wir auch, wie sich der verwendete Code auf die Länge eines Bitstreams auswirkt.

Als erstes codieren wir das Datum als String.

In [ ]:
b = BinaryStream()

# codieren
for c in "09.12.2018":
    b.writeChar(c)
print("Bitstream: ", b)
print("Länge: ", b.length())

# decodieren
while not b.isEmpty():
    print(b.readChar(), end="")
print()


Eine weitere Variante wäre, Tag, Monat und Jahr jeweils als Integer zu speichern. 

In [ ]:
b = BinaryStream()

# codieren
b.writeInt(9)
b.writeInt(12)
b.writeInt(2018)

print("Bitstream", b)
print("Länge", b.length())

# decodieren
print(b.readInt(), b.readInt(), b.readInt())

In der letzten Variante, die auch die effizienteste ist, codieren wir Tag/Monat/Jahr mit nur jeweils sovielen Bits wie dafür benötigt werden.

In [ ]:
b = BinaryStream()

# codieren
b.writeNBitNumber(9, 5)
b.writeNBitNumber(12, 4)
b.writeNBitNumber(2018, 11)

print("Bitstream: ", b)
print("Länge: ", b.length())

# decodieren
print(b.readNBitNumber(5), b.readNBitNumber(4), b.readNBitNumber(11))


## Lauflängen Codierung

Die erste, einfachste Art der Komprimierung ist die Lauflängen Codierung. 
Dieser Code ist vor allem dann nützlich, wenn wir lange Sequenzen von aufeinanderfolgenden 0 oder 1 haben. 
Als Testbeispiel generieren wir einen Bitstring, der aus 100 0 gefolgt von 50 1 besteht. 

In [ ]:
testStream = BinaryStream()
for _ in range(0, 100):
    testStream.writeBit(False)
for _ in range(0, 50):
    testStream.writeBit(True)
print(testStream)

Die Kompressionsfunktion ersetzt dann jede Sequenz von 0 und 1 durch die entsprende Anzahl.

In [ ]:
def compress(stream):
    compressedStream = BinaryStream()
    old = False
    count = 0 
    for b in stream:
        if b != old:
            compressedStream.writeNBitNumber(count, 8)
            count = 0
            old = not old
        else:
            if count == 255:
                compressedStream.writeNBitNumber(255, 8)
                count = 0
                compressedStream.writeNBitNumber(0, 8) # dummy count of length 0 to indicate bit switch
        count += 1
    compressedStream.writeNBitNumber(count, 8) 
    return compressedStream
        

Wenn wir diese Funktion auf unseren Bitstring aufrufen, sehen wir, dass dieser stark komprimiert ist. 

In [ ]:
compressedStream = compress(testStream)
print(compressedStream)
print("Länge: ", compressedStream.length())

Natürlich können wir diesen auch wieder entpacken. 

In [ ]:
def expand(compressedStream):
    stream = BinaryStream()
    b = False
    while not compressedStream.isEmpty():
        c = compressedStream.readNBitNumber(8)
        for i in range(0, c):
            stream.writeBit(b)
        b = not b
    return stream
            

In [ ]:
print(expand(compressedStream))

# Huffman

Bei der Huffman Codierung wird der Code dynamisch, basierend auf der Häufigkeit der einzelnen Buchstaben, aufgebaut. Dies passiert mit Hilfe eines binären Tries. Da wir diesen nur intern verwenden, implementieren wir diesen direkt als verkettete Datenstruktur von Nodes, die wie folgt definiert sind:

In [ ]:
class Node:
    def __init__(self, char, frequency, left, right):
        self.left = left
        self.right = right
        self.char = char
        self.frequency = frequency
        
    # Vergleichsmethode, wird gebraucht da wir die Nodes in einer 
    # Priorityqueue, basierend auf der Frequenz, speichern wollen.
    def __lt__(self, rhs):
        return self.frequency < rhs.frequency
        
    def __str__(self):
        return str((self.char, self.frequency))

Die Funktion buildTrie bestimmt die Auftrittshäufigkeit von jedem Buchstaben und baut basierend darauf den Trie auf.

In [ ]:
import heapq 

def buildTrie(alphabet, message):
    freqs = [0] * alphabet.radix()
    
    for c in message:
        freqs[alphabet.toIndex(c)] += 1

    pq = []
    for i in range(0, alphabet.radix()):
        if freqs[i] > 0:
            heapq.heappush(pq, Node(alphabet.toChar(i), freqs[i], None, None))
    heapq.heapify(pq)
    while len(pq) > 1:    
        n1 = heapq.heappop(pq)
        n2 = heapq.heappop(pq)
        
        heapq.heappush(pq, Node(None, n1.frequency + n2.frequency, n1, n2))
    return pq[0]


In [ ]:
codeTrie = buildTrie(ascii, "she sells sea shells by the sea")
plotTrie(codeTrie)

Basierend auf dem Trie erhalten wir nun einfach den Code für jeden Buchstaben.

In [ ]:
def buildCode(root):
    st = dict(); 
    _buildCode(st, root, "")
    return st;

def _buildCode(st, node, code):
    if node.left == None and node.right == None: # Wir sind an einem Blatt
        st[node.char] = code
        return; 
    
    _buildCode(st, node.left,  code + '0') 
    _buildCode(st, node.right, code + '1')


In [ ]:
codeTable = buildCode(codeTrie)
print(codeTable)

Nachdem wir nun die Codetabelle haben, ist es auch kein Problem mehr eine Nachricht zu codieren. 

In [ ]:
def compressWithCode(codeTable, message, bitstream):
    for c in message:
        code = codeTable[c]
        for b in code:
            bitstream.writeBit(True if b == '1' else False)
    return
        

In [ ]:
bitString = BinaryStream()
compressWithCode(codeTable, "she sells sea shells by the sea", bitString)
print(bitString)

Gleich einfach ist es, diesen Code wieder zu entpacken. Wir folgen einfach den Zeichen im Trie

In [ ]:
def expandWithCode(codeTrie, bitstring):
    s = ""
    while not bitstring.isEmpty():
        x = codeTrie;       
        while x.left != None or x.right != None: # noch nicht am Blatt
            b = bitstring.readBit()
            if b == True:
                x = x.right;          
            else:
                x = x.left;       
        # an Blatt
        s = s + x.char
    return s

In [ ]:
expandWithCode(codeTrie, bitString)

#### Trie schreiben und lesen

Nachdem wir nun einen String komprimieren und dekomprimieren können, gibt es nur noch ein Problem zu lösen. Wie kommt der Trie mit den Codes zum Empfänger.

Die folgende Methode schreibt einen Trie als Bitstream

In [ ]:
def writeTrie(node, bitstream):
    
    if node.left == None and node.right == None:
        bitstream.writeBit(True)
        bitstream.writeChar(node.char)
        return; 
    
    bitstream.writeBit(False) 
    writeTrie(node.left, bitstream) 
    writeTrie(node.right, bitstream)

In [ ]:
bsForTrie = BinaryStream()
writeTrie(codeTrie, bsForTrie)
print(bsForTrie)


Die Methode zum lesen ist sogar noch einfacher (*zu schreiben, aber nicht zu verstehen*).

In [ ]:
def readTrie(bitstream): 
    if bitstream.readBit() == True:
        c = bitstream.readChar()
        return Node(c, 0, None, None); 
    return Node(None, 0, readTrie(bitstream), readTrie(bitstream))

In [ ]:
plotTrie(readTrie(bsForTrie))

Zusammenfassend, kann man die Methode zum Komprimieren und Dekomprimieren eines Strings wie folgt schreiben

In [ ]:
def compress(message):
    bs = BinaryStream()
    codeTrie = buildTrie(ascii, message)
    codeST = buildCode(codeTrie)
    writeTrie(codeTrie, bs)
    compressWithCode(codeST, message, bs)
    return bs

In [ ]:
def expand(bs):
    codeTrie = readTrie(bs)
    return expandWithCode(codeTrie, bs)

In [ ]:
bs = compress("she sells sea shells by the sea")
print("Bitstream: ", bs)
print("Länge", bs.length())
print("dekomprimiert: ", expand(bs))